In [ ]:
import sys
import os

notebook_cwd = os.getcwd()
src_path = os.path.join(notebook_cwd, '..')
if src_path not in sys.path:
    sys.path.append(src_path)


# Imports

In [ ]:
from pprint import pprint

import numpy as np
import matplotlib.pyplot as plt

from src.interface.configuration import load_config

%matplotlib inline


In [ ]:
def print_attributes(cls, title: str = "Class Attributes"):
    print(f"\n{title}")
    for attribute, path in cls.__dict__.items():
        print(f"- {attribute}: {path}")


# Loading project configuration

In [ ]:
config = load_config()
directories_paths = config.directory_paths
json_paths = config.database_paths

print(f"\nProject Path: {directories_paths.project}")
print_attributes(cls=directories_paths, title="Directory Paths:")    
print_attributes(cls=json_paths, title="Database JSON File Paths:")   

# Opening / Loading the Database

In [ ]:
database = config.database()

print("Database \"Tables\":")
for attribute in database.__dict__.keys():
    print(f"- {attribute}")
    

## Loading an interferometric device

An interferometric device is represented by the ```Interferometer``` class children:
* Michelson Interferometer: ```MichelsonInterferometer```
* Fabry-Perot Interferometer: ```FabryPerotInterferometer```

Each interferometric device is characterized  / parametrized by the following parameters:
* Transmittance coefficients (polynomial coefficients): ```transmittance_coefficients```
* Reflectivity coefficients (polynomial coefficients): ```reflectance_coefficients```
* Optical Path Differences (OPD): ```opds```
* Phase shifts: ```phase_shift```

It is possible to store the parameters of an interferometric device in two ways:
* By directly registering their values in the ```JSON``` file (useful for simulations and easy / free manipulations), which is ```interferometers.json```. In this case, it is also possible to freely choose the harmonic_order of a Fabry-Perot device.
* By storing the values in ```.npy``` files (useful when loading characterizations parametrized from real devices), which is ```characterizations.json``` [1].

After that, the database is used to generate an ```Interferometer``` object.

In the database, it is specified whether an interferometer record is of type Michelson or Fabry-Perot.

*[1] Picone, Daniele, et al. "Interferometer response characterization algorithm for multi-aperture Fabry-Perot imaging spectrometers." Optics Express 31.14 (2023): 23066-23085.*

### Loading from the interferometers option

In [ ]:
interferometer_id = 4

interferometer = database.interferometer(interferometer_id=interferometer_id)

plt.plot(interferometer.opds)

### Loading from the characterizations option

In [ ]:
characterization_id = 0

characterization = database.characterization(characterization_id=characterization_id)
interferometer = characterization.interferometer()

plt.plot(interferometer.opds)

## Loading a dataset

A dataset can be loaded using the dataset id.

There are two categories of datasets:
* Spectrum datasets
* Interferogram datasets

Spectrum datasets are represented by the ```Spectrum``` class, containing the following attributes:
* Data: ```data```
* Corresponding wavenumbers: ```wavenumbers```
* Wavenumbers unit: ```wavenumbers_unit```

Interferogram datasets are represented by the ```Interferogram``` class, containing the following attributes:
* Data: ```data```
* Corresponding Optical Path Differences (OPDs): ```opds```
* OPDs unit: ```opds_unit```

The following code snippet is used to load a dataset from the database.
The method returns a ```Spectrum``` or an ```Interferogram``` based on the category of the dataset record in the database.

In [ ]:
dataset_id = 1

dataset = database.dataset(dataset_id=dataset_id)

print(f"\nDataset shape: {dataset.data.shape}")

fig, axs = plt.subplots(1, 1, squeeze=False)
dataset.visualize(axs=axs[0, 0], acq_ind=13)

In [ ]:
dataset_id = 3

dataset = database.dataset(dataset_id=dataset_id)

print(f"\nDataset shape: {dataset.data.shape}")

fig, axs = plt.subplots(1, 1, squeeze=False)
dataset.visualize(axs=axs[0, 0], acq_ind=13)

It is also possible to load the central wavenumbers directly:
* For spectrum datasets, this simply represents the corresponding wavenumbers samples.
* For interferogram datasets, if exists, this refers to the wavenumbers used in the instrument line shape of the device [1].

*[1] Picone, Daniele, et al. "Interferometer response characterization algorithm for multi-aperture Fabry-Perot imaging spectrometers." Optics Express 31.14 (2023): 23066-23085.*

In [ ]:
dataset_id = 1

wavenumbers = database.dataset_central_wavenumbers(dataset_id=dataset_id)

print(f"Central wavenumbers length: {wavenumbers.size}")

## Loading an inversion protocol

The inversion protocol is represented by the ```InversionProtocol``` class.
The supported inversion protocol types are:
* The Inverse Discrete Cosine Transform (IDCT): ```IDCT```
* The Moore-Penrose inverse (pseudo-inverse): ```PseudoInverse```
* The Truncated Singular Value Decomposition (TSVD): ```TSVD```
* The Tikhonov regularization / Ridge Regression (RR): ```RidgeRegression```
* The Loris-Verhoeven (LV) algorithm: ```LorisVerhoeven```

In the database, a range of regularization parameters is provided for each inversion protocol record (the number of parameters can also be one). Thus, generally speaking, when loading the inversion protocol from the database, a list of the ```InversionProtocol``` class objects is provided, of the same inversion protocol type, each with a different regularization parameter.

In [ ]:
inv_protocol_id = 2

inv_protocols = database.inversion_protocol_list(inv_protocol_id=inv_protocol_id)

print(f"Number of inversion protocols: {len(inv_protocols)}")
print(f"Type of inversion protocol: {type(inv_protocols[0])}")

It is also possible to:
* Load an inversion protocol with a specific regularization parameter.
* Load only the array of regularization parameters.

In [ ]:
inv_protocol_id = 2

reg_param = 0.6
inv_protocol = database.inversion_protocol(inv_protocol_id=inv_protocol_id, lambdaa=reg_param)
print(f"\nInversion Protocol with a chosen regularization parameter:")
print(f"- Type: {type(inv_protocol)}")
print(f"- Info: {inv_protocol}")

reg_params_array = database.inversion_protocol_lambdaas(inv_protocol_id=inv_protocol_id)
print(f"\nRange of regularization parameters:\n{np.round(reg_params_array, decimals=3)}")